In [1]:
import pandas as pd

In [2]:
file =open('sample','r', errors='ignore')
file2 =open('replaced','w+')
contents=file.read()
for x in range(len(contents)):

    if contents[x] == ",":
        file2.write("!!@@@")
    else:
        file2.write(str(contents[x]))
file2.close()

In [3]:
names=["Month" ,"Day" ,"Time" , "Machine", "Application"]
df= pd.read_csv('replaced', sep=" " , usecols=range(5) ,names=names)

In [4]:
df["Application"] = df["Application"].replace({':':''}, regex=True)

In [5]:
file3 =open('replaced','r', errors='ignore')
file4 =open('last','w+')
contents2=file3.read()
for x in range(len(contents2)):
    if contents2[x] == ":" and contents2[x-1] != "d":
        file4.write("!-!")
    else:
        file4.write(str(contents2[x]))
file4.close()

In [6]:
df2 = pd.read_csv('last', header=None, sep='\n' , encoding='latin-1')
df2 = df2[0].str.split(': ', expand=True)
df2["Info"]=df2[df2.columns[1:]].apply(lambda x: ': '.join(x.dropna().astype(str)),axis=1)


In [7]:
Structure = pd.DataFrame([df["Month"],df["Day"],df["Time"],df["Machine"],df["Application"]]) 
Structure=Structure.T

In [8]:
final=pd.concat([Structure ,df2["Info"]],axis=1)


The next cell changes the value of month to numbers

In [9]:
import datetime
for key, value in final["Month"].iteritems():
    month_name = value
    datetime_object = datetime.datetime.strptime(month_name, "%b")
    month_number = datetime_object.month
    final["Month"][key]=month_number


In [10]:
final["Date"]=final.apply(lambda final: datetime.datetime(2020,final["Month"],final['Day']), axis=1)

for key, value in final["Date"].iteritems():
    final["Date"]=datetime.datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S').strftime('%d-%m-%y %H:%M:%S')


In [11]:
for key, value in final["Date"].iteritems():
    out=value.split(' ')
    final["Date"]= out[0]

In [12]:
final=final.drop(columns=["Month","Day"])

In [13]:
cols = final.columns.tolist()
cols = cols[-1:] + cols[:-1]
final=final[cols]

In [14]:
Application_Values=final["Application"].value_counts()

In [15]:
Application_Values

setsid    322
Name: Application, dtype: int64

In [16]:
final["ind"]= final["Info"].str.find("failed") 

In [17]:
final["a"]=final["Info"].apply(lambda x: 1 if "failed" in str(x).lower() or "Failed" in str(x).lower() else 0)

In [18]:
sum=final["a"].sum()
final=final.drop(columns=["ind","a"])
sum

163

In [19]:
final

,Date,Time,Machine,Application,Info
0,11-07-20,12:09:10,dhcp31371,setsid,Loaded: loaded (/usr/lib/systemd/system/etcd.s...
1,11-07-20,12:09:10,dhcp31371,setsid,Active!-! failed (Result!-! start-limit) since...
2,11-07-20,12:09:10,dhcp31371,setsid,Main PID!-! 14281 (code=exited!!@@@ status=1/F...
3,11-07-20,12:09:10,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Fa...
4,11-07-20,12:09:10,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Un...
...,...,...,...,...,...
317,11-07-20,12:09:32,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Fa...
318,11-07-20,12:09:32,dhcp31371,setsid,Jul 11 11!-!27!-!48 dhcp31371 systemd[1]!-! Un...
319,11-07-20,12:09:32,dhcp31371,setsid,Jul 11 11!-!27failed:48 dhcp31371 systemd[1]!-...
320,11-07-20,12:09:33,dhcp31371,setsid,starting etcd=0
